# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 16 2022 11:02AM,244253,10,MSP213131,"Methapharm, Inc.",Released
1,Aug 16 2022 11:02AM,244253,10,MSP213139,"Methapharm, Inc.",Released
2,Aug 16 2022 11:02AM,244250,10,MSP213132,"Methapharm, Inc.",Released
3,Aug 16 2022 11:02AM,244250,10,MSP213149,"Methapharm, Inc.",Released
4,Aug 16 2022 11:02AM,244250,10,MSP213152,"Methapharm, Inc.",Released
5,Aug 16 2022 11:01AM,244251,22,624412,"NBTY Global, Inc.",Released
6,Aug 16 2022 10:59AM,244249,10,MSP213126,"Methapharm, Inc.",Released
7,Aug 16 2022 10:59AM,244249,10,MSP213127,"Methapharm, Inc.",Released
8,Aug 16 2022 10:59AM,244249,10,MSP213128,"Methapharm, Inc.",Released
9,Aug 16 2022 10:59AM,244249,10,MSP213129,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
31,244247,Released,16
32,244249,Released,21
33,244250,Released,3
34,244251,Released,1
35,244253,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
244247,NaN,16.0
244249,NaN,21.0
244250,NaN,3.0
244251,NaN,1.0
244253,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
244147,30.0,15.0
244164,0.0,1.0
244183,0.0,1.0
244189,0.0,1.0
244191,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
244147,30,15
244164,0,1
244183,0,1
244189,0,1
244191,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,244147,30,15
1,244164,0,1
2,244183,0,1
3,244189,0,1
4,244191,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,244147,30,15
1,244164,,1
2,244183,,1
3,244189,,1
4,244191,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 16 2022 11:02AM,244253,10,"Methapharm, Inc."
2,Aug 16 2022 11:02AM,244250,10,"Methapharm, Inc."
5,Aug 16 2022 11:01AM,244251,22,"NBTY Global, Inc."
6,Aug 16 2022 10:59AM,244249,10,"Methapharm, Inc."
27,Aug 16 2022 9:59AM,244222,15,"Virtus Pharmaceuticals, LLC"
39,Aug 16 2022 9:58AM,244243,20,"Exact-Rx, Inc."
44,Aug 16 2022 9:58AM,244247,10,Eywa Pharma Inc.
60,Aug 16 2022 9:41AM,244242,10,Beach Patient Assistance
61,Aug 16 2022 9:37AM,244241,10,"Nextsource Biotechnology, LLC"
62,Aug 16 2022 9:30AM,244240,10,"CLINUVEL, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Aug 16 2022 11:02AM,244253,10,"Methapharm, Inc.",,2
1,Aug 16 2022 11:02AM,244250,10,"Methapharm, Inc.",,3
2,Aug 16 2022 11:01AM,244251,22,"NBTY Global, Inc.",,1
3,Aug 16 2022 10:59AM,244249,10,"Methapharm, Inc.",,21
4,Aug 16 2022 9:59AM,244222,15,"Virtus Pharmaceuticals, LLC",6,6
5,Aug 16 2022 9:58AM,244243,20,"Exact-Rx, Inc.",,5
6,Aug 16 2022 9:58AM,244247,10,Eywa Pharma Inc.,,16
7,Aug 16 2022 9:41AM,244242,10,Beach Patient Assistance,,1
8,Aug 16 2022 9:37AM,244241,10,"Nextsource Biotechnology, LLC",,1
9,Aug 16 2022 9:30AM,244240,10,"CLINUVEL, Inc.",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 16 2022 11:02AM,244253,10,"Methapharm, Inc.",2,
1,Aug 16 2022 11:02AM,244250,10,"Methapharm, Inc.",3,
2,Aug 16 2022 11:01AM,244251,22,"NBTY Global, Inc.",1,
3,Aug 16 2022 10:59AM,244249,10,"Methapharm, Inc.",21,
4,Aug 16 2022 9:59AM,244222,15,"Virtus Pharmaceuticals, LLC",6,6
5,Aug 16 2022 9:58AM,244243,20,"Exact-Rx, Inc.",5,
6,Aug 16 2022 9:58AM,244247,10,Eywa Pharma Inc.,16,
7,Aug 16 2022 9:41AM,244242,10,Beach Patient Assistance,1,
8,Aug 16 2022 9:37AM,244241,10,"Nextsource Biotechnology, LLC",1,
9,Aug 16 2022 9:30AM,244240,10,"CLINUVEL, Inc.",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 16 2022 11:02AM,244253,10,"Methapharm, Inc.",2,
1,Aug 16 2022 11:02AM,244250,10,"Methapharm, Inc.",3,
2,Aug 16 2022 11:01AM,244251,22,"NBTY Global, Inc.",1,
3,Aug 16 2022 10:59AM,244249,10,"Methapharm, Inc.",21,
4,Aug 16 2022 9:59AM,244222,15,"Virtus Pharmaceuticals, LLC",6,6


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Aug 16 2022 11:02AM,244253,10,"Methapharm, Inc.",2.0,NaN
1,Aug 16 2022 11:02AM,244250,10,"Methapharm, Inc.",3.0,NaN
2,Aug 16 2022 11:01AM,244251,22,"NBTY Global, Inc.",1.0,NaN
3,Aug 16 2022 10:59AM,244249,10,"Methapharm, Inc.",21.0,NaN
4,Aug 16 2022 9:59AM,244222,15,"Virtus Pharmaceuticals, LLC",6.0,6.0


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 16 2022 11:02AM,244253,10,"Methapharm, Inc.",2.0,0.0
1,Aug 16 2022 11:02AM,244250,10,"Methapharm, Inc.",3.0,0.0
2,Aug 16 2022 11:01AM,244251,22,"NBTY Global, Inc.",1.0,0.0
3,Aug 16 2022 10:59AM,244249,10,"Methapharm, Inc.",21.0,0.0
4,Aug 16 2022 9:59AM,244222,15,"Virtus Pharmaceuticals, LLC",6.0,6.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3419303,147.0,2.0
15,732653,80.0,44.0
16,976797,4.0,0.0
19,488370,1.0,1.0
20,976799,22.0,30.0
21,732583,3.0,0.0
22,244251,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3419303,147.0,2.0
1,15,732653,80.0,44.0
2,16,976797,4.0,0.0
3,19,488370,1.0,1.0
4,20,976799,22.0,30.0
5,21,732583,3.0,0.0
6,22,244251,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,147.0,2.0
1,15,80.0,44.0
2,16,4.0,0.0
3,19,1.0,1.0
4,20,22.0,30.0
5,21,3.0,0.0
6,22,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,147.0
1,15,Released,80.0
2,16,Released,4.0
3,19,Released,1.0
4,20,Released,22.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21,22
Status,,,,,,,
Executing,2.0,44.0,0.0,1.0,30.0,0.0,0.0
Released,147.0,80.0,4.0,1.0,22.0,3.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21,22
0,Executing,2.0,44.0,0.0,1.0,30.0,0.0,0.0
1,Released,147.0,80.0,4.0,1.0,22.0,3.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21,22
0,Executing,2.0,44.0,0.0,1.0,30.0,0.0,0.0
1,Released,147.0,80.0,4.0,1.0,22.0,3.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()